### Import Necessary Libraries

In [1]:
import joblib
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

C:\Users\kommi\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Import reuquired extra requirments

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kommi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kommi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kommi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

set the stop words to not iterate again and again in for loop later

In [3]:
stop_words = set(stopwords.words('english'))

intialise lemmatizer

In [4]:
lem=WordNetLemmatizer()

### Load trained models 

In [5]:
# Load the trained models
loaded_spam_detect_model_NB = joblib.load('spam_detect_model_NB.joblib')
loaded_spam_detect_model_svm = joblib.load('spam_detect_model_svm.joblib')
loaded_cv = joblib.load('tfidf_vectorizer.joblib')

In [6]:
new_messages = ["Get a free iPhone now!", "Congratulations, you've won a prize! Claim now!"]

### Pre-processing Text

In [7]:
# Preprocess the new data
new_corpus = []
for i in range(0, len(new_messages)):
    review = re.sub('[^a-zA-Z]', ' ', new_messages[i])
    review = review.lower()
    review = review.split()
    review = [lem.lemmatize(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    new_corpus.append(review)

In [8]:
print(new_corpus)

['get free iphone', 'congratulation prize claim']


#### Using bag of word mechanism to vectorize the words

In [9]:
# Transform the new data using the loaded TF-IDF vectorizer
new_X = loaded_cv.transform(new_corpus).toarray()

#### Predict outcome of classification of each sentence

In [10]:
new_y_pred_NB = loaded_spam_detect_model_NB.predict(new_X)
print("Predictions using Multinomial NB Model:", ["spam" if label == 1 else "ham" for label in new_y_pred_NB])

# Test the SVM model
new_y_pred_svm = loaded_spam_detect_model_svm.predict(new_X)
print("Predictions using SVM Model:", ["spam" if label == 1 else "ham" for label in new_y_pred_svm])

Predictions using Multinomial NB Model: ['ham', 'spam']
Predictions using SVM Model: ['ham', 'spam']


After each E-mail is separated into multiple sentences , models are predicting if that particular sentences are spam or not , but no final output

Using both models and thresold values to classify if E-mail is spam if it has more than 30% of its sentences classified as spam by ml models

In [11]:
# Combine predictions for each sentence
combined_predictions = ['spam' if (new_y_pred_NB[i] == 1 or new_y_pred_svm[i] == 1) else 'ham' for i in range(len(new_y_pred_NB))]

Calculate the percentage of sentences classified as spam

In [12]:
percentage_spam = combined_predictions.count('spam') / len(combined_predictions)

Classify the entire paragraph or email based on the percentage threshold

In [13]:
final_classification = 'spam' if percentage_spam >= 0.3 else 'ham'

print("Final Classification:", final_classification)

Final Classification: spam


#### CODE BY : @AJAY